In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression ,  Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingRegressor , ExtraTreesRegressor , AdaBoostRegressor
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss, accuracy_score
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.compose import TransformedTargetRegressor
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV 
from sklearn.linear_model import LinearRegression
import datetime 
import calendar

import warnings 
warnings.filterwarnings('ignore')

# Import and Modifying Data

In [52]:
df = pd.read_csv(r'C:\Users\hp\Desktop\ONGC\train.csv', sep = ';')
df

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,34.636410,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,34.636410,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,34.779556,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,39.965164,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,57.483765,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,7/1-2 S,3169.3124,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.423170,NaN,NaN,...,27.674368,NaN,NaN,-0.001763,NaN,NaN,26.673708,NaN,30000,2.0
1170507,7/1-2 S,3169.4644,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.379244,NaN,NaN,...,28.024338,NaN,NaN,-0.007600,NaN,NaN,26.840818,NaN,65030,2.0
1170508,7/1-2 S,3169.6164,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.350248,NaN,NaN,...,28.091282,NaN,NaN,-0.018297,NaN,NaN,27.007942,NaN,65030,2.0
1170509,7/1-2 S,3169.7684,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.313779,NaN,NaN,...,28.019775,NaN,NaN,-0.011438,NaN,NaN,27.175179,NaN,65030,2.0


In [53]:
new_df = df.drop(['RSHA', 'SGR', 'ROP', 'DTS', 'DCAL', 'MUDWEIGHT', 'RMIC', 'ROPA', 'RXO'], axis=1)
new_df

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RMED,RDEP,RHOB,GR,NPHI,PEF,DTC,SP,BS,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,1.611410,1.798681,1.884186,80.200851,NaN,20.915468,161.131180,24.612379,NaN,-0.574928,65000,1.0
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,1.618070,1.795641,1.889794,79.262886,NaN,19.383013,160.603470,23.895531,NaN,-0.570188,65000,1.0
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,1.626459,1.800733,1.896523,74.821999,NaN,22.591518,160.173615,23.916357,NaN,-0.574245,65000,1.0
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,1.621594,1.801517,1.891913,72.878922,NaN,32.191910,160.149429,23.793688,NaN,-0.586315,65000,1.0
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,1.602679,1.795299,1.880034,71.729141,NaN,38.495632,160.128342,24.104078,NaN,-0.597914,65000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,7/1-2 S,3169.3124,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.423170,NaN,NaN,2.527984,77.654900,NaN,4.586425,NaN,NaN,8.5,-0.001763,30000,2.0
1170507,7/1-2 S,3169.4644,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.379244,NaN,NaN,2.537613,75.363937,NaN,7.019858,NaN,NaN,8.5,-0.007600,65030,2.0
1170508,7/1-2 S,3169.6164,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.350248,NaN,NaN,2.491860,66.452843,NaN,9.049782,NaN,NaN,8.5,-0.018297,65030,2.0
1170509,7/1-2 S,3169.7684,NaN,NaN,NaN,VESTLAND GP.,Bryne Fm.,8.313779,NaN,NaN,2.447539,55.784817,NaN,8.903917,NaN,NaN,8.5,-0.011438,65030,2.0


In [54]:
dfna = new_df.dropna()
dfna = dfna.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)

In [55]:
dfna = dfna[['RHOB', 'NPHI', 'DTC', 'GR', 'DRHO', 'FORCE_2020_LITHOFACIES_LITHOLOGY']]
dfna

,RHOB,NPHI,DTC,GR,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY
22641,2.063168,0.541850,134.226379,68.388885,0.026319,65000
22642,2.051136,0.545401,134.824799,71.170013,0.019921,65000
22643,2.041540,0.548953,135.037079,74.497498,0.014932,65000
22644,2.035698,0.549356,134.500336,76.098335,0.009946,65000
22645,2.029099,0.543351,132.162399,74.966675,0.005706,65000
...,...,...,...,...,...,...
1070829,2.498846,0.227478,75.882454,86.000305,0.026406,65030
1070830,2.498926,0.228153,75.577896,84.984161,0.024520,65030
1070831,2.497899,0.232037,74.278587,85.125671,0.022548,65030
1070832,2.496484,0.241528,74.009476,88.016106,0.018891,65030


Using Label Encoder to encode all values of Output to integer

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_encoded = le.fit_transform(dfna.FORCE_2020_LITHOFACIES_LITHOLOGY)
dfna['Litho_encoded'] = y_encoded
dfna.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True, errors='ignore')

In [7]:
dfna.to_csv('preprocessed_data.csv', index=None)

In [8]:
df = pd.read_csv('preprocessed_data.csv')
df.head()

,RHOB,NPHI,DTC,GR,DRHO,Litho_encoded
0,2.063168,0.541850,134.226379,68.388885,0.026319,1
1,2.051136,0.545401,134.824799,71.170013,0.019921,1
2,2.041540,0.548953,135.037079,74.497498,0.014932,1
3,2.035698,0.549356,134.500336,76.098335,0.009946,1
4,2.029099,0.543351,132.162399,74.966675,0.005706,1


In [9]:
df['Litho_encoded'].value_counts()

1    90958
0    23249
2    14108
3     8597
6     4928
4     1941
8     1328
9      852
5      186
7       20
Name: Litho_encoded, dtype: int64

In [13]:
forecast = pd.read_csv('test.csv', sep=';')
forecast = forecast[['RHOB', 'NPHI', 'DTC', 'GR','DRHO']]
forecast.dropna(inplace=True)
forecast.head()

,RHOB,NPHI,DTC,GR,DRHO
0,2.171173,0.551575,174.347092,72.078377,-0.538873
1,2.171840,0.547566,176.112915,72.147697,-0.539232
2,2.161303,0.551840,177.662476,75.074219,-0.540830
3,2.151947,0.552511,177.437714,80.735229,-0.543943
4,2.143010,0.566789,176.797653,81.665024,-0.542104


In [14]:
X = df.drop('Litho_encoded', axis=1)
y = df.Litho_encoded
Y = pd.DataFrame(y)

scaler = StandardScaler()
X=scaler.fit_transform(X)
forecast = scaler.fit_transform(forecast)
X = pd.DataFrame(X)
forecast = pd.DataFrame(forecast)

In [15]:
folds = StratifiedKFold(n_splits=5,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])
    
    wrapped_model = SVC() 
    wrapped_model.fit(*train_set)
    
    preds = wrapped_model.predict(test_set[0])
    score = accuracy_score(test_set[1], preds)
      
    oof_pred_fold = wrapped_model.predict(forecast)
    oof_preds.append(oof_pred_fold)

    print(score)
    oof_scores.append(score)
    print(f"Iteration number {i} completed")

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)
#final_preds = pd.DataFrame(final_preds)
final_preds = final_preds.round(0)
final_preds=final_preds.astype(int)
final_preds = le.inverse_transform(final_preds)
final_preds = pd.DataFrame(final_preds)
final_preds.to_csv('predictions.csv', index=None)

0.8028592927012792
Iteration number 0 completed
0.8063346559036804
Iteration number 1 completed
0.806355613326948
Iteration number 2 completed
0.8066028053369826
Iteration number 3 completed
0.8057750863868076
Iteration number 4 completed
Mean OOF Score : 0.8055854907311396


# Thus the SVM model has been trained with average accuracy of 80.559%

In [16]:
#p.unique(final_preds)
final_preds.shape

(94520, 1)

In [ ]:
#final_preds=final_preds.astype(int)
#final_preds = le.inverse_transform(final_preds)
#final_preds = pd.DataFrame(final_preds)
#final_preds.to_csv('predictions.csv', index=None)

In [ ]:
final_preds.iloc[:,0].value_counts()

In [25]:
le.inverse_transform(wrapped_model.predict([[2.171173,0.551575,174.347092,72.078377,-0.538873]]))

array([65000], dtype=int64)

In [ ]:
# a = classification_report(test_set[1],preds)
# a

In [45]:
import pickle
filename = 'final_model.pkl'
pickle.dump(wrapped_model, open(filename, 'wb'))

In [46]:
loaded_model = pickle.load(open('final_model.pkl', 'rb'))

In [47]:
le.inverse_transform(loaded_model.predict([[2.171173,0.551575,174.347092,72.078377,-0.538873]]))

array([65000], dtype=int64)

In [49]:
loaded_model.predict([[2.171173,0.551575,174.347092,72.078377,-0.538873]])

array([1], dtype=int64)

The trained model will return a value between 0-9 which is going to correspond with target variable
FORCE_2020_LITHOFACIES_LITHOLOGY as follows:
30000: 0
65000: 1
65030: 2
70000: 3
70032: 4
74000: 5
80000: 6
88000: 7
90000: 8
99000: 9

These values will correspond to the lithology of that particular sample as follows:
30000: Sandstone
65000: Shale
65030: Sandstone/Shale
70000: Limestone
70032: Chalk
74000: Dolomite
80000: Marl
88000: Halite
90000: Coal
93000: Basement